# Complex TFIDF


## A review of TFIDF

(Skip ahead if you're familiar with TF-IDF)

TFIDF is an embedding algorithm that converts a list of documents into vectors whose components correspond to the terms comprising each document and the values of the components reflect the frequency of that terms in the document as well as how common it is over all documents. By a "term"", we could mean words or other units of text like bigrams or ngrams, so splitting "hello world" by words would return tokens "hello" and "world", whereas splitting by bigrams would return "he","el","ll",...,"ld". 

Let's recall the definition of TFIDF explicitly. Consider a list of $N$ documents $d_1,...,d_N$, where each document $d_i$ is comprised of a subset of terms from a set of distinct terms $T$ (the dictionary), each term $t\in T$ appearing $tf_i(t)$ in the document $d_i$ (the *term frequency*) and appearing in $N(t)$ documents. If we enumerate the dictionary $T$ as $t_{1},...,t_{D}$, then the Term Frequency-Inverse Document Frequency (TF-IDF) representation of the document $d_i$ is the vector in $\mathbb{R}^{D}$ whose $i$th component is

$$ tf_{i}(t)\log\frac{N}{N(t)}.$$

(Note that one can also normalize $tf_i(t)$ to be the proportion of times $t$ appears in document $i$ rather than the number, but we will work with this definition.)

This represents a document in such a way that the dominant components correspond to terms that appear frequently in the document that are rare overall while suppressing the values of very common terms. Imagining the documents in question are pages of wikipedia, components corresponding to terms like "the" and "is" will have small values since these terms are in most every page, so the inverse document frequency $N/N(t)$ will be close to $1$ and hence the logarithm will be close to zero, whereas terms like "Mingus" and "heteroskadicity" are quite rare overall, making the logarithm much larger. 

Of course, there are more sophisticated text embeddings that encode context and meaning of a document (noteably BERT and its siblings), so that "The War of the Worlds" and "The Battle of the Planets" will have close representations (at least more so in comparison to unrelated sentences) than they would with a TF-IDF representation. In particular, the order of terms matter, so the representations of "the dog caught the ball" and "the ball caught the dog" would be different, whereas the TF-IDF representations would be identical. However, TF-IDF is a lightweight, transparent representation that is still very useful.

TF-IDF is also good at representing short strings when using bigrams (or ngrams) as the terms rather than words: now the components of the embeddings correspond to bigrams (so for "amazon", the nonzero components will correspond to "am","ma","az",...). One reason is that it can be forgiving of alternate spellings of words (e.g. "amazoncom", "amazon.com", "amazon com" will have similar representations). 

## Adding order to TF-IDF wtih complex numbers.

One question I thought about was whether one could come up with a TF-IDF-like representation that somehow encodes the position of the terms, but without having to use something as complex as Attention or Transformers. A cheap way of doing this is using complex numbers. 

Let's assume all our documents have length at most $M$, that is, each document $d_i$ is an ordered list of terms $t_{i,1},...,t_{i,m_i}$ (where $t_{i,j}\in T$ and $m_i\leq M$ and the terms may repeat. If $e_{i,j}$ is the standard basis vector corresponding to the term $t_{i,j}$, we define a complex TF-IDf representation of $d_i$ as 

$$CTFIDF(d_i) = \sum_{j=1}^{m_i} e^{\frac{\pi ij}{4M}} \log\frac{N}{N(t_{i,j})}$$

Some remarks

1. Without the complex exponential this would just return the usual TFIDF. 
2. Note that this is effectively a vector in $\mathbb{R}^{D}\times i\mathbb{R}^{D}\equiv \mathbb{R}^{2D}$, so we have only doubled the dimension of the representation.
3. It is not hard to show that, if each term appears exactly once in $d_i$, then $|TFIDF(d_i)| = |CTFIDF(d_i)|$, however now it is possible to
 have strings with identical TFIDF representation but distinct CTFIDF. 

# Demonstration

Below we use a custom tfidf object that can do both traditional and complex tfidf. As a demonstration, we fit it on the words in the above text.




In [6]:
import pandas as pd
import re
from tfidf import TFIDF
import numpy as np



In [ ]:
tfidf = TFIDF(gramlen=2)

In [2]:
d = open("text.txt","r").read()
d = re.findall("[a-z]{2,}",d)
df = pd.DataFrame({"text":d})
tfidf.fit(df,"text")

100%|██████████| 605/605 [00:00<00:00, 11650.26it/s]


The tfidf object has a 'show' method that shows the unnormalized tfidf representation of a word. Note that "ja" is out of dictionary and doesn't appear in the representation. 

In [3]:
tfidf.show("llama llama red pajama")

(' l', 6.921578957728802)
('ll', 6.721412040614837)
('la', 9.591581091193483)
('am', 11.760965424728523)
('ma', 10.897919207373182)
('a ', 17.13624383241269)
(' r', 3.004031076368686)
('re', 2.185720752854735)
('ed', 4.208003880694622)
('d ', 2.794310545386617)
(' p', 4.459318308975528)
('pa', 4.795790545596741)


We can also view the complex tfidf representation. Below instead of having a number for each component, we have the real and complex part. 


In [4]:
tfidf.show_complex("llama llama red pajama")

(' l', 6.544375483375044, 1.571165545003015)
('ll', 6.215819018726448, 2.019642027403319)
('la', 8.616622458672762, 3.569121884230861)
('am', 6.793039094674124, 7.381548106389901)
('ma', 5.738496254087156, 7.312653123404562)
('a ', 8.093408638945665, 12.171175626881828)
(' r', 1.765725164117798, 2.430312192412732)
('re', 1.1420359562364624, 1.8636333019462112)
('ed', 1.9103937847025037, 3.749358911508878)
('d ', 1.0693363506025169, 2.5816063203631465)
(' p', 1.378005140800791, 4.2410637359854615)
('pa', 1.1195550688935425, 4.663282471065833)


Note above that " p" and "pa", since they are further along in the string, have smaller real part than they do imaginary, whereas terms appearing mostly in the first half of the string like "ll" have larger real part than imaginary.  

# A comparison of representations in clustering

We demonstrate how the complex tfidf can outperform the usual tfidf using some synthetic data. Synthetic data isn't ideal, but this is just a POC and maybe at a future point we could use some realworld data for a better test. 

The way we evaluate this is by testing how well KMeans clustering works using each representation. We do this by creating some text consisting of some made up business names plus some noise (random words, think of the strings as transactions consiting of the business name plus an order number), label the strings by their business name, then apply KMeans with the same number of clusters as business names.

Ideally, we'd expect a good clustering to return the original groups, that is, the strings that appear in each cluster all belong to one business. So a clustering that results in clusters where 90% of strings in each cluster correspond to one business might be called good. However, imagine we have two clustering that are like this, but in one clustering the remaining 10% in each cluster comes from one other business and in the other clustering the remaining 10% come from two other businesses. It would seem that the second clustering is more confused about grouping transactions together than the first, as the first is at best confusing two categories with each other in each cluster, not three. 

A better way of evaluating the quality of how good a clustering performs on labeled data is to look at the weighted average of the entropies of the labels in each cluster. That is, if We have N strings with labels $l_1,...,l_n$, and we cluster into $m$ clusters $C_1,...,C_m$ of sizes $N_1,...,N_m$, we consider the quantity:

$$ \sum_{i=1}^{m} H_i \frac{N_i}{M}$$

where 

$$H_i = \sum_{j=1}^{n} p_j\log \frac{1}{p_j}$$

and 

$$p_i=\frac{|\{x\in C_i: x\in l_j\}|}{|C_i|}.$$





In [10]:
#Create the synthetic data

import string
alphabet = list(string.ascii_lowercase)

def random_text(N=10):
    return "".join(np.random.choice(alphabet,N))
random_text(10)

#We make up 4 business names, intentionally choosing names that are quite similar
#but are likely to be confused by TFIDF, in the sense that the sets of bigrams will have 
#many common elements.

a = "jacks of london"
b = "jack and jones"
c = "jones of london"
d = "london jacks"

#We create 100 distinct strings for each business, consisting of the business name
#plus some noise word. We also create a vector with just the business name.
X = []
y = []
n=100
for z in [a,b,c,d]:
    y = y + [z]*n
    X = X + [z + " " +  random_text() for _ in range(n)]
    
#The TFIDF object is designed for working with dataframes, so we put the 
#text and labels into one.
d = pd.DataFrame({"text":X, "label":y})
    
d.sample(20)


,text,label
392,london jacks xxjmmqffip,london jacks
162,jack and jones lpjrpokpgr,jack and jones
6,jacks of london kavjdewaum,jacks of london
122,jack and jones wklmptqkgf,jack and jones
127,jack and jones idpbpwfwwe,jack and jones
264,jones of london oqsazygzla,jones of london
341,london jacks jacwonqnfe,london jacks
194,jack and jones wtpewnbnus,jack and jones
89,jacks of london dilqkggoab,jacks of london
310,london jacks jypxitsuqp,london jacks


In [14]:
tfidf.fit(d,"text")

100%|██████████| 400/400 [00:00<00:00, 7096.72it/s]


In [16]:
#When we cluster our data using either the tfidf or ctfidf representations, we'll
#add the labels to the dataframe. We give here some functions to compute the entropy
#of the cluster labels using the dataframe.

from scipy.stats import entropy

def entropy_of_cluster(g, label_col):
    label_counts = g[label_col].value_counts()
    distribution = label_counts / label_counts.sum()
    return entropy(distribution)
    

def average_entropy_of_clusters(df, label_col, cluster_col):
    n_samples = df.shape[0]
    return  df.groupby(cluster_col).apply(lambda g:entropy_of_cluster(g, label_col)*g.shape[0]).sum()/n_samples

In [20]:
#Now we cluster using Kmeans and 4 clusters using both embeddigns.

from sklearn.cluster import KMeans

#TFIDF
KM = KMeans(4)
KM.fit(tfidf.transform(d.text))
d["tfidf_clusters"] = KM.labels_

#CTFIDF
KM = KMeans(4)
KM.fit(tfidf.complex_transform(d.text))
d["complex_tfidf_clusters"] = KM.labels_

d

100%|██████████| 400/400 [00:00<00:00, 4266.62it/s]


,text,label,tfidf_clusters,complex_tfidf_clusters
0,jacks of london zawbeoyakp,jacks of london,2,3
1,jacks of london lahjnidayh,jacks of london,3,3
2,jacks of london vjqyknyzdi,jacks of london,2,3
3,jacks of london yldefxajch,jacks of london,2,3
4,jacks of london fwplhexbbs,jacks of london,2,3
...,...,...,...,...
395,london jacks iggrjhhzfz,london jacks,2,0
396,london jacks fdtxtyxbog,london jacks,2,0
397,london jacks raisjljyik,london jacks,2,0
398,london jacks dylccruzeh,london jacks,2,0


In [21]:
for embedding in ["tfidf","complex_tfidf"]:
    print(embedding, average_entropy_of_clusters(d,"label",f"{embedding}_clusters"))



tfidf 0.3568763571081831
complex_tfidf 0.0


We see that the complex tf_idf results in a perfect labeling of the data, whereas clustering using the tfidf embedding results in some confusion. 